In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
import numpy as np
import re
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "/content/drive/MyDrive/생성 AI 모델링/data/"
# DATA_PATH = "/content/drive/MyDrive/멀티캠퍼스 자료/Machine Learning/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device




'cuda'

# 데이터 정제

In [ ]:
yogiyo =pd.read_csv(f"{DATA_PATH}yogiyo_reviews_jsi_all.csv")


In [ ]:
google_food = pd.read_csv(f"{DATA_PATH}playstore_food_reply_0904.csv")

In [ ]:
paraphrased = pd.read_csv(f'{DATA_PATH}train_over.csv')

## 정규표현식

In [ ]:
train= yogiyo[['고객리뷰','사장댓글']]

rename_cols = ['review', 'reply']
train.columns = rename_cols
train.head()

In [ ]:
review_list = []
for review in train['review']:
    if isinstance(review, str):

        pat = re.compile("[^a-zA-Z가-힣0-9.,~?!]")
        result = pat.sub(" ", review)

        result = re.sub(r'\.{2,}', '.', result)
        result = re.sub(r'\,{2,}', ',', result)
        result = re.sub(r'\~{2,}', '~', result)
        result = re.sub(r'\!{2,}', '!', result)
        result = re.sub(r'\?{2,}', '?', result)
        result = re.sub(" +", " ", result)
        review_list.append(result)
    else:
        review_list.append("")

train['review'] = review_list
train.head()

In [ ]:
reply_list = []
for reply in train['reply']:
    if isinstance(reply, str):

        pat = re.compile("[^a-zA-Z가-힣0-9.,~?!]")
        result = pat.sub(" ", reply)

        result = re.sub(r'\.{2,}', '.', result)
        result = re.sub(r'\,{2,}', ',', result)
        result = re.sub(r'\~{2,}', '~', result)
        result = re.sub(r'\!{2,}', '!', result)
        result = re.sub(r'\?{2,}', '?', result)
        result = re.sub(" +", " ", result)
        reply_list.append(result)
    else:
        reply_list.append("")

train['reply'] = reply_list
train.head()

## 1) 고객id 처리
## - ~님 -> 고객님

In [ ]:
train['reply'] = train['reply'].str.replace(r'\w+\s*님', '고객님', regex=True)


In [ ]:

for review in train['reply']:
  if '고객님' in review:
      print(review)
      break

## 2) 가게명 처리
- reply에 가게명이 들어가있는 경우에 store 칼럼에 가게명 기재, 모델이 함께 학습할 수 있도록 처리
- '저희 가게'로 처리?



In [ ]:
ygy_store_lst = ["1988응답하라추억의옛날도시락-방이점", "워커스커피 로스터스", "떡깨비-가락점", "당치땡-헬리오시티점", "우시장국밥-석촌점",  "고돼지-송파점",  "마라의신마라탕&마라샹궈",  "춘리마라탕-송파점",  "손수경의육회담은연어-가락점",  "무국적식탁-송파점",  "청년치킨-삼전점",  "송파만다린",  "배달의쌀국수-송파점",  "닭강점-헬리오시티점",  "토핑폭탄김치찜&김치찌개-송파점",  "조형훈족발보쌈",  "여왕의브런치",  "모어댄버터",  "플러스82-송파점",  "더티베지",  "잭아저씨족발보쌈-본점",  "육회&연어-미아본점",  "갈비민족-본점",  "구구족-성신여대역점",  "피자보이시나-대학로점",  "왕빈자삼파전-미아본점",  "행복한찜닭-성북점",  "스시사소우",  "저팔계&중국&마라탕&덮밥&요리&전문점",  "명작파스타-성북점",  "올데이파스타-동대문구점",  "대한냉면-성북점",  "인생제육",  "고기듬뿍미트박스-성북점",  "피자스웨그-성북점",  "청춘식당묵은지김치찜",  "삽교원조두리곱창",  "뜸들이다-북서울꿈의숲점",  "더팔당매운오징어&닭발 김치찜",  "우리집반찬도시락-혜화막내딸지점",  "수피자",  "마라쿡",  "김포불닭발",  "헬로팬닭갈비&마약볶음밥",  "대찌 대파생고기김치찌개&정육왕국물두루치기",  "구래상회",  "승도리네곱도리탕",  "자성당",  "수락",  "나인곱창",  "어시장",  "돈까스회관",  "호랑이초밥",  "땡초곱창막창",  "피자이탈리",  "텐텐마라탕",  "과일에반하다",  "삼다수",  "병천토속순대",  "최고남제육명가",  "짱닭치킨",  "걸작떡볶이",  "직화삼겹직구삼",  "짜글이가돼었소",  "부대찌개자신있는집",  "토핑폭탄김치찜&김치찌개-부천점",  "1989마라탕-부천점",  "미친피자-부평점",  "천년닭강정-부천부평점",  "수상한삼겹살-부천점",  "수제죽전문점-본가진죽-본점",  "오늘은 분식",  "이끌림마라탕",  "탐나는피자-부평점",  "고기혁명-부천점",  "라화방마라탕-신중동점",  "전금례닭볶음탕-부천점",  "큰아들백순대",  "공주닭발-부천시청점",  "신자매김치찜&김치찌개",  "짱닭치킨-도촌점",  "이태리면가게-야탑점",  "폴트버거-판교점",  "랜돌프뉴욕페페로니-야탑점",  "홍싸롱-수제돈까스&파스타",  "쇼부덮밥",  "카산도-정자본점",  "부성초밥-미금본점",  "닭장수섭삼계탕&전기구이통닭",  "깨돌이김밥-미금점",  "울진죽변항",  "GTS버거-분당정자점",  "코브라독스-분당미금역점",  "마라하오-죽전점",  "땅끝수산",  "초밥대통령-용인수지점",  "올데이케밥&샐러드",  "야키토리 카마쿠라",  "비오키친-서현본점",  "부산오빠1인국밥-분당점",  "알촌",  "밈피자",  "고씨네카레",  "홍스족발",  "뜸들이다",  "감성커피",  "짚신스시",  "떡형",  "39도시락",  "피제이피자",  "황궁쟁반짜장",  "OK반점",  "모모타코야키&카페",  "꼬알라파이",  "진지덮밥",  "피자프라텔로",  "한식세끼1인김치찜&김치찌개",  "윤희횟집",  "시민보쌈족발&감자탕",  "카페봄봄",  "킹프레소빅와플",  "최가네한쌈",  "창타이누들",  "보돌미역",  "홍대개미"  ]


In [ ]:
store_name = []

for store in ygy_store_lst:
    pat = re.split(r'[- &]', store)
    store_name.append(pat[0])

len(store_name)

In [ ]:
store_name[:20]

In [ ]:
# train['store'] = ""  # full store name을 칼럼에 담기
# for name in store_name:
#     train[name] = train['reply'].apply(lambda x: name if name in x else '') # 우선 각 가게명 컬럼 생성(그렇게해야 덮어쓰기가 안됨)
#     train['store'] = train.apply(lambda row: row['store'] + ',' + row[name] if row[name] else row['store'], axis=1) # 하나로 합치기(맨 앞에 콤마가 생김)

# train['store'] = train['store'].apply(lambda x: x[1:] if x.startswith(',') else x) # 맨 앞에 콤마 제거
# train = train.drop(columns=store_name) # 각 가게명 컬럼 삭제

# train.head()

[링크 텍스트](https:// [링크 텍스트](https://))- 1차 : store 이름이 풀로 들어간 경우 <store>로 대체

In [ ]:
def replace_store_names1(text):
    for name1 in store_name:
        if name1 in text:
            # 가게 이름의 일부가 텍스트에 포함되어 있다면 대체
            pattern = re.compile(re.escape(name1), re.IGNORECASE)
            text = pattern.sub("<store>", text)
    return text

In [ ]:
replace_store_names1("1988응답하라추억의옛날도시락에서 먹은 음식은 정말 맛있었습니다.")

In [ ]:
replace_store_names1("킹프레소에서 먹은 음식은 정말 맛있었습니다.") # 안바뀜

- 2차 : store 이름이 부분적으로 들어간 경우 store로 대체

In [ ]:
def replace_store_names2(text):
    for name1 in tqdm(store_name, desc="Processing"):
        for i in range(len(name1), 1, -1):
            part = name1[:i]  # 가게 이름에서 처음부터 i 글자까지의 부분
            if part in text:
                # 가게 이름의 일부가 텍스트에 포함되어 있다면 대체
                pattern = re.compile(re.escape(part), re.IGNORECASE)
                text = pattern.sub("<store>", text)
    return text


In [ ]:
replace_store_names2("텐텐에서 먹은 음식은 정말 맛있었습니다.") # 텐텐마라탕

In [ ]:
replace_store_names2("손수경에서 먹은 음식은 정말 맛있었습니다.") # 손수경의육회담은연어

- 3차 : 일반명사 등은 제외하고 대체해야 함

In [ ]:
!pip install kiwipiepy

In [ ]:
# # short_name 에서 가게명 이외 일상용어로 사용되는 키워드는 제거
# drop_name = ['고기','OK','감성','걸작','공주','과일','김포','대한','땡초','명작','미친','배달','병천','부대','부산','비오','삽교',
#              '손수','송파','쇼부','수상','수제','시민','오늘', '오늘은', '우리', '울진', '이끌', '이끌림', '인생', '직화', '진지',
#              '천년', '청년', '청춘', '최고', '큰아', '큰아들', '탐나', '토핑', '행복', '행복한', '헬로', '홍대',
#              '육회','수락','호랑','갈비','무국','닭장','한식','마라','스시','여왕','돈까','초밥','카페','피자','부대찌','이태리','플러','플러스']
# # drop_name = ['과일','오늘', '오늘은','우리', '이끌', '이끌림', '인생','진지','최고''토핑', '행복', '행복한', '한식', '감성','걸작']

In [ ]:
import kiwipiepy
kiwi = kiwipiepy.Kiwi()

In [ ]:
def remove_adjectives_nouns(text):
    result = []
    tokens = kiwi.extract_words(text, min_cnt=1, min_score=0.25)
    for token in tokens:
        word = token[0].form  # 단어만 추출하여 사용
        pos = token[1]
        # 형용사나 명사가 아닌 경우에만 결과에 추가
        if pos not in ('VA', 'NNG'):
            result.append(word)
    return ' '.join(result)


In [ ]:
tokens = kiwi.analyze(store_name)

remove_words = []

for token in tokens :
  word = token[0][0][0].form
  pos = token[0][0][0].tag
  if pos in ('NNG') and len(word) ==2 :
    print(word)
    remove_words.append(word)

# 2글자 일반명사 단어들은 제외 -> 완벽하지 않아서 세부적인 튜닝 필요. 대찌, 구래, 삼다 등

In [ ]:
tokens = kiwi.analyze(store_name)

for token in tokens :
  word = token[0][0][0].form
  pos = token[0][0][0].tag
  if pos in ('NNG') and len(word) ==3 :
    print(word)

# 3글자 일반명사 단어들은 가게명으로 포함해도될듯

In [ ]:
tokens = kiwi.analyze(store_name)

for token in tokens :
  word = token[0][0][0].form
  pos = token[0][0][0].tag
  if pos in ('NNG') and len(word) ==4 :
    print(word)

# 4글자 일반명사 단어들은 가게명으로 포함해도될듯

In [ ]:
store_name[:5]

- 최종 합수

In [ ]:
# def replace_store_names(text):
#     for name1 in store_name:
#         # 먼저 가게 이름 전체가 텍스트에 포함되어 있는지 확인
#         if name1 in text:
#             # 가게 이름 전체가 텍스트에 포함되어 있다면 대체
#             pattern = re.compile(re.escape(name1), re.IGNORECASE)
#             text = pattern.sub("<store>", text)
#         else:
#             # 가게 이름의 일부가 텍스트에 포함되어 있는 경우를 확인
#             for i in range(len(name1), 1, -1):
#                 part = name1[:i]  # 가게 이름에서 처음부터 i 글자까지의 부분
#                 if part in text:
#                     # 가게 이름의 일부가 텍스트에 포함되어 있다면 대체
#                     pattern = re.compile(re.escape(part), re.IGNORECASE)
#                     text = pattern.sub("<store>", text)
#                     break  # 가게 이름 일부를 대체했으면 루프 종료
#     return text


In [ ]:
def replace_store_names(text):
    for name1 in store_name:
        # 먼저 가게 이름 전체가 텍스트에 포함되어 있는지 확인
        if name1 in text:
            # 가게 이름 전체가 텍스트에 포함되어 있다면 대체
            pattern = re.compile(re.escape(name1), re.IGNORECASE)
            text = pattern.sub("<store>", text)
        else:
            # 가게 이름의 일부가 텍스트에 포함되어 있는 경우를 확인
            for i in range(len(name1), 1, -1):
                part = name1[:i]  # 가게 이름에서 처음부터 i 글자까지의 부분
                if part in text:
                    # 가게 이름의 일부가 텍스트에 포함되어 있고, 해당 부분이 removelist에 없는 경우에만 대체
                    if part not in remove_words:
                        pattern = re.compile(re.escape(part), re.IGNORECASE)
                        text = pattern.sub("<store>", text)
                    break  # 가게 이름 일부를 대체했으면 루프 종료
    return text


In [ ]:
remove_words[:10]

In [ ]:
korean_particles = ['은', '는', '이', '가', '을', '를', '의', '한', '로', '으로', '까지', '이며', '며', '와', '과']

#위 일반명사에, 조사를 합친 경우도 제외함

In [ ]:
remove_withparticles = []

for word in remove_words :
  for particle in korean_particles:
    new_word = word + particle
    remove_withparticles.append(new_word)

In [ ]:
def replace_store_names(text):
    for name1 in store_name:
        # 먼저 가게 이름 전체가 텍스트에 포함되어 있는지 확인
        if name1 in text:
            # 가게 이름 전체가 텍스트에 포함되어 있다면 대체
            pattern = re.compile(re.escape(name1), re.IGNORECASE)
            text = pattern.sub("<store>", text)
        else:
            # 가게 이름의 일부가 텍스트에 포함되어 있는 경우를 확인
            for i in range(len(name1), 1, -1):
                part = name1[:i]  # 가게 이름에서 처음부터 i 글자까지의 부분
                if part in text:
                    # 가게 이름의 일부가 텍스트에 포함되어 있고, 해당 부분이 remove_words에 없으며,
                    # 해당 부분 뒤에 조사가 붙지 않은 경우에만 대체
                    if part not in remove_words + remove_withparticles :
                        pattern = re.compile(re.escape(part), re.IGNORECASE)
                        text = pattern.sub("<store>", text)
                    break  # 가게 이름 일부를 대체했으면 루프 종료
    return text


In [ ]:
replace_store_names('행복한 라화방마라탕에서 맛있게 식사했어요') #행복 부분이 대체안됨

In [ ]:
# train['reply']의 답변들에 함수 적용하기 / 18000개 7분 정도 걸림

for i, review in tqdm(enumerate(train['reply']), desc="Processing", ncols=100):
    train['reply'][i] = replace_store_names(review)

In [ ]:
changed_rows = train[train['reply'].str.contains("<store>")] #총 9446개
changed_rows

In [ ]:
# train['review']의 리뷰들에 함수 적용하기 / 7분 정도 걸림

for i, review in tqdm(enumerate(train['review']), desc="Processing", ncols=100):
    train['review'][i] = replace_store_names(review)

In [ ]:
changed_rows = train[train['review'].str.contains("<store>")] #총 25개
changed_rows

In [ ]:
# len(set(train['reply'].tolist())) # 우선은 중복제거 안함. 어떤 유형의 리뷰에 대해 동일답변하는 하는지 체크 필요

In [ ]:
store_mask = []

for review in train['reply']:
  if '<store>' in review:
      print(review)
      break

In [ ]:
# store_mask = []   # 확인용

# for review in train['reply']:
#   if '<store>' not in review:
#       print(review)

In [ ]:
# short_store_name = []
# for name in store_name:
#     for length in [2, 3, 4,5,6]:
#         cut = name[:length]
#         short_store_name.append(cut)
#     # if name[0].isdigit(): # 숫자로 시작하면 텍스트가 나오는 부분까지 추출
#     #     cut = name[:6]
#     #     short_name.append(cut)

# short_store_name = [name for name in short_store_name if not name.isdigit()] # 숫자로만 되어 있는 키워드 삭제
# short_store_name = list(set(short_store_name))
# print(len(short_store_name), short_store_name)



In [ ]:
# # short_name 에서 가게명 이외 일상용어로 사용되는 키워드는 제거
# drop_name = ['고기','OK','감성','걸작','공주','과일','김포','대한','땡초','명작','미친','배달','병천','부대','부산','비오','삽교',
#              '손수','송파','쇼부','수상','수제','시민','오늘', '오늘은', '우리', '울진', '이끌', '이끌림', '인생', '직화', '진지',
#              '천년', '청년', '청춘', '최고', '큰아', '큰아들', '탐나', '토핑', '행복', '행복한', '헬로', '홍대',
#              '육회','수락','호랑','갈비','무국','닭장','한식','마라','스시','여왕','돈까','초밥','카페','피자','부대찌','이태리','플러','플러스']
# # drop_name = ['과일','오늘', '오늘은','우리', '이끌', '이끌림', '인생','진지','최고''토핑', '행복', '행복한', '한식', '감성','걸작']

# # 명사인 경우에만 남기기 형용사, 부사 등이
# # 청년 치킨

# short_store_name = [col for col in short_store_name if col not in drop_name]
# print(len(short_store_name),short_store_name)

In [ ]:
# store_name = store_name + short_store_name  # 두 그룹은 set처리 안함. store name(full)이 우선적으로 처리되어야 하기 때문

In [ ]:
# len(store_name)

In [ ]:
# def replace_store_names(text, store_name):
#     for store in store_name:
#         text = text.replace(store, "<store>")
#     return text

# for store in store_name:
#     for i, review in enumerate(train['reply']):
#         if store in review:
#             train['reply'][i] = replace_store_names(review, [store])

In [ ]:
# mask = train['store'] != ''
# train[mask]    #약 1/3 6330개 리뷰답변에 가게명 확인 / 글자 2~7 추가 -> 11379 리뷰

## 3) 지점명 처리
- 전국일반음식점 표준 데이터 : https://www.data.go.kr/data/15096283/standard.do
- 행정구역 리스트 -> 리스트 내에 있는 단어인 경우 확인해서 region 칼럼에 추가 or 삭제
- 지점명의 경우 보통 가게명과 함께 등장하기 때문에, 빈칸으로 삭제해도 무방

- ~호점 삭제

In [ ]:
# 정규 표현식을 사용하여 "~호점" 패턴을 찾아서 삭제하는 함수
def remove_hojum(text):
    return re.sub(r'\w+\s*호점','', text)

# train 데이터프레임의 'reply' 열에 적용하여 "~호점"을 삭제
train['reply'] = train['reply'].apply(remove_hojum)
train['review'] = train['review'].apply(remove_hojum)


In [ ]:
mask = []

for review in train['review']:
  if '호점' in review:
      print(review)


In [ ]:
# restaurant_seoul = pd.read_csv(f"{DATA_PATH}소상공인시장진흥공단_상가(상권)정보_서울_202212.csv")

In [ ]:
restaurant_list = pd.read_csv(f"{DATA_PATH}fulldata_07_24_04_P_일반음식점.csv",encoding='cp949')

<ipython-input-4-afda4487c689>:1: DtypeWarning: Columns (11,27,39,41,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  restaurant_list = pd.read_csv(f"{DATA_PATH}fulldata_07_24_04_P_일반음식점.csv",encoding='cp949')


In [ ]:
restaurant_list['소재지전체주소']

0                   서울특별시 동대문구 장안동 382-1
1               서울특별시 마포구 망원동 386-2번지 지층
2                 서울특별시 마포구 서교동 403-14번지
3              서울특별시 마포구 서교동 363-5번지 B1동
4          경기도 안양시 동안구 호계동 930-30번지 지상1층
                       ...              
2086169             광주광역시 서구 용두동 306번지 1
2086170              광주광역시 남구 양림동 374 2층
2086171     광주광역시 남구 월산동 1048-182번지 (1층)
2086172        광주광역시 서구 화정동 332-9번지 (1층)
2086173          광주광역시 서구 화정동 70-61 (1층)
Name: 소재지전체주소, Length: 2086174, dtype: object

In [ ]:
address_list = pd.DataFrame(restaurant_list['소재지전체주소'])

In [ ]:
address_list.to_csv(f"address_list.csv")

In [ ]:
# 가게명에서 지점명 추출하기

store_etc_name = []

for store in ygy_store_lst:
    pat = re.split(r'[- &]', store)
    for name in pat[1:]:
        if name != '':
            store_etc_name.append(name)

ygy_region_name = []

for region in store_etc_name:
  if region[-1] =='점':
    ygy_region_name.append(region)

NameError: ignored

In [ ]:
address_parts=[]

for address in restaurant_list['소재지전체주소'].astype(str):
    if address.startswith('서울'or '부산'):
        parts = address.split(' ')
        for _ in range(0, min(4, len(parts))):  # 최대 5개 또는 주소 부분 개수만큼 반복
          part = parts[_]
          address_parts.append(part)

address_parts = set(address_parts)
address_parts = list(address_parts)

address_parts = [name for name in address_parts if all(char.isalpha() for char in name)] #알파벳으로만 (한글) 구성된 경우남김. 숫자, 특수문자 지움
address_parts.remove('')

In [ ]:
address_parts2 = [] # 추출된 지역명을 지점명으로 변경

for address in address_parts :
  if len(address) < 3  :
    address = address + '점'
    address_parts2.append(address)
  else :
    address = address[:-1] + '점'
    address_parts2.append(address)

In [ ]:
len(address_parts2)

In [ ]:
region_name = ygy_region_name + address_parts2

In [ ]:
region_name = set(region_name)
region_name = list(region_name)

In [ ]:
region_name[:10]

- 지점명의 경우 1, 2, 3차로 나누지 않고 지점명 풀로 포함된 경우에만 대체 (~점 포함, 기본 2글자)

In [ ]:
def replace_region_names(text):
    for name1 in region_name:
        if name1 in text:
            # 지점 이름이 텍스트에 포함되어 있다면 대체
            pattern = re.compile(re.escape(name1), re.IGNORECASE)
            text = pattern.sub("<region>", text)
        # else :
        #   for i in range(len(name1), 1, -1):
        #       part = name1[:i]  # 지점 이름에서 처음부터 i 글자까지의 부분
        #       if part in text:
        #           # 지점 이름의 일부가 텍스트에 포함되어 있다면 대체
        #           pattern = re.compile(re.escape(part), re.IGNORECASE)
        #           text = pattern.sub("<region>", text)
    return text


In [ ]:
replace_region_names('오랜만에 헬리오시티점에서 맛있게 식사했어요')

In [ ]:
for i, review in tqdm(enumerate(train['reply']), desc="Processing", ncols=100):
    train['reply'][i] = replace_region_names(review)

In [ ]:
changed_rows = train[train['reply'].str.contains("<region>")]
changed_rows

In [ ]:
for i, review in tqdm(enumerate(train['review']), desc="Processing", ncols=100):
    train['review'][i] = replace_region_names(review)

In [ ]:
changed_rows = train[train['review'].str.contains("<region>")]
changed_rows

In [ ]:
region_mask = []

for review in train['reply']:
  if '<region>' in review:
      print(review)
      break

In [ ]:
train.isna().sum()

## 4) 메뉴명 처리
-
- 일반 메뉴 리스트 -> 리스트 내에 있는 단어인 경우 확인해서 menu 칼럼에 추가 or 삭제
- 특수 메뉴 리스트 -> ex. 저희 ~~커피, 저희 ~~찜 등

In [ ]:
food_list = pd.read_csv(f'{DATA_PATH}전국통합식품영양성분정보(음식)표준데이터.csv', encoding='cp949')

In [ ]:
food_list = set(food_list['대표식품명'].tolist())

In [ ]:
food_list = list(food_list)

In [ ]:
ygy_menu_list = ['양식','일식','중식','중국집','태국음식','한식','분식','도시락','카레','커피','국밥','마라탕','마라샹궈','우동','나가사키','연어','치킨','쌀국수','닭강정','김치찜','김치찌개',
                '족발','보쌈','육회','갈비','피자','찜닭','파스타','냉면','곱창','닭발','막창','반찬','곱도리탕','순대','떡볶이','부대찌개'
                '삼겹살','분식','라면','버거','덮밥','초밥','통닭','짜장','짬뽕','탕수육','카페','와플','누들','김치','온면','디저트','라떼','고기','덮밥',
             '새우','돼지','닭','찌개','밥','타코야키',
                 '양파','숙주','고수'] # 재료명 제거

In [ ]:
menu_name = food_list + ygy_menu_list

In [ ]:
len(menu_name)

In [ ]:
new_food_list = []  # 새로운 음식을 담을 리스트

for food in food_list:
    if '(' in food:
        food1, food2 = food.split('(')
        new_food_list.append(food1.strip())  # 공백 제거 후 리스트에 추가
        new_food_list.append(food2.rstrip(')'))
    else:
        new_food_list.append(food)

food_list = new_food_list

In [ ]:
menu_name = set(menu_name)
menu_name = list(menu_name)

menu_name = [name for name in menu_name if all(char.isalpha() for char in name)] #알파벳으로만 (한글) 구성된 경우남김. 숫자, 특수문자 지움


- 1차 : 풀로 이름 들어간 것 menu로 대체

In [ ]:
def replace_menu_names1(text):
    # 메뉴 이름을 긴 것부터 짧은 것 순으로 정렬
    sorted_menu_names = sorted(menu_name, key=len, reverse=True)

    for name1 in sorted_menu_names:
        if name1 in text:
            # 메뉴 이름이 텍스트에 포함되어 있다면 대체
            pattern = re.compile(re.escape(name1), re.IGNORECASE)
            text = pattern.sub("<menu>", text)
    return text


In [ ]:
replace_menu_names1('부대찌개 정말 맛있었어요')

- 2차 : 부분 이름 들어간 것 대체

In [ ]:
menu_name[:10]

In [ ]:
name1='단무지무침'

for i in range(len(name1),1,-1):
    part = name1[:i]
    print(part)

In [ ]:
def replace_menu_names2(text):

    for name1 in menu_name:
        for i in range(len(name1), 1, -1):
            part = name1[:i]  # 가게 이름에서 처음부터 i 글자까지의 부분
            if part in text:
                # 가게 이름의 일부가 텍스트에 포함되어 있다면 대체
                pattern = re.compile(re.escape(part), re.IGNORECASE)
                text = pattern.sub("<menu>", text)

    return text


In [ ]:
replace_menu_names2('부대 정말 맛있었어요')

- 3차 : 일반명사 제외 안해도됨. 재료명 모두 일반명사

- 최종 함수

In [ ]:
def replace_menu_names(text):
    # 메뉴 이름을 긴 것부터 짧은 것 순으로 정렬
        # sorted_menu_names = sorted(menu_name, key=len, reverse=True)
    for name1 in menu_name:
        if name1 in text:
            # 메뉴 이름이 텍스트에 포함되어 있다면 대체
            pattern = re.compile(re.escape(name1), re.IGNORECASE)
            text = pattern.sub("<menu>", text)
        else:
            for i in range(len(name1), 1, -1):
                part = name1[:i]  # 가게 이름에서 처음부터 i 글자까지의 부분
                if part in text:
                    # 가게 이름의 일부가 텍스트에 포함되어 있다면 대체
                    pattern = re.compile(re.escape(part), re.IGNORECASE)
                    text = pattern.sub("<menu>", text)
                    break  # 대체를 하고 나서는 루프를 종료합니다.
    return text


In [ ]:
train.isna().sum()

In [ ]:
for i, review in tqdm(enumerate(train['reply']), desc="Processing", ncols=100):
    train['reply'][i] = replace_menu_names(review)

In [ ]:
train['reply']

In [ ]:
changed_rows = train[train['reply'].str.contains("<menu>")]
changed_rows

In [ ]:
for menu in menu_name:
    for i, review in enumerate(train['review']):
        if menu in review:
            train['review'][i] = replace_menu_names(review)

In [ ]:
changed_rows = train[train['review'].str.contains("<menu>")]
changed_rows

In [ ]:
train.replace(['<menu><menu>','<menu> <menu>'], '<menu>')

## 5) 시간, 장소, 매장번호, 슬로건 등 고유 정보 처리
- 48시간, 24시간,02, 031, 번호 등

In [ ]:
train.to_csv(f'{DATA_PATH}train_token_masked.csv',index=False)

## 맞춤법

In [ ]:
!git clone https://github.com/ssut/py-hanspell.git

In [ ]:
!cd py-hanspell

In [ ]:
!pip install py-hanspell/

In [ ]:
from hanspell import spell_checker

def correct_spell(text):
    if isinstance(text, str):  # 입력 데이터가 문자열인 경우에만 맞춤법 검사 수행
        corrected_text = spell_checker.check(text).checked
        return corrected_text
    else:
        return text

In [ ]:
correct_spell(train['review'][0])

In [ ]:
train.head()

In [ ]:
correct_spell('감사합니다!!! 다음에도 이용해주세용')

In [ ]:
train['review'][:10].apply(correct_spell)

In [ ]:
train['reply'][:10].apply(correct_spell)

In [ ]:
# tqdm.pandas().  # 오래걸려서 우선 모델 학습시에 맞춤법 처리 된걸 넣을 필요는 없을 것 같고, 답변을 내뱉을때 맞춤법 처리 기능을 함수에 추가하기로

# train['review'] = train['review'].progress_apply(correct_spell)
# train['reply'] = train['reply'].progress_apply(correct_spell)

In [ ]:
# train.to_csv(f'{DATA_PATH}train_posneg_grammarchecked.csv',index=False)

# 데이터 분석/증강/변형

## 긍부정 비율 확인
- 식당 별점 평가 사전학습 모델 : MLOpsEngineer/review-rate-prediction
- Label_0 = 1점, Label_1 = 2점, Label_2 = 3점, Label_3 = 4점, Label_4 = 5점


In [ ]:
from transformers import pipeline

# 모델 불러오기
model_name = "MLOpsEngineer/review-rate-prediction"
classifier = pipeline("sentiment-analysis", model=model_name)


In [ ]:
classifier = pipeline("sentiment-analysis", model=model_name)

In [ ]:
print(train['review'][0])
classifier(train['review'][0])

In [ ]:
# 분류할 리뷰 텍스트
review_text = "맛있어요!"

# 모델 추론
result = classifier(review_text)[0]

# 결과 해석
label = result['label']
score = result['score']

if label in ['LABEL_0','LABEL_1'] :   # 어떤 기준으로 label 1, 2, 3을 잡을지 확인 필요. 대략은 0,1만 부정으로 하는게 좋아보임
    sentiment = '부정'
else:
    sentiment = '긍정'

# 출력
print(f"리뷰의 감정: {sentiment}")
print(f"긍부정 지수: {score}")


In [ ]:
import pandas as pd
from transformers import pipeline

def analyze_reviews(reviews):
    # 모델 불러오기
    model_name = "MLOpsEngineer/review-rate-prediction"
    classifier = pipeline("sentiment-analysis", model=model_name)

    results = []
    for review_text in tqdm(reviews):
        # 모델 추론
        result = classifier(review_text)[0]

        # 결과 해석
        label = result['label']
        score = result['score']

        if label in ['LABEL_0', 'LABEL_1']:
            sentiment = '부정'
        else:
            sentiment = '긍정'

        results.append((review_text, sentiment, score))

    # 데이터프레임 생성
    df = pd.DataFrame(results, columns=['review', 'sentiment', 'score'])
    return df

In [ ]:
# train_posneg = analyze_reviews(train['review'])

In [ ]:
# train_posneg

In [ ]:
# train_posneg.to_csv(f"{DATA_PATH}train_posneg.csv",index=False)
train_posneg = pd.read_csv(f"{DATA_PATH}train_posneg.csv")

In [ ]:
train_posneg

In [ ]:
cols = ['review','Unnamed: 0']
train_posneg = train_posneg.drop(columns=cols)
train_posneg

In [ ]:
train = pd.concat([train,train_posneg],axis=1,ignore_index=True)


In [ ]:
# train.to_csv(f"{DATA_PATH}train_posneg_done.csv",index=False)

In [ ]:
train.columns = ['review','reply','sentiment','score']

In [ ]:
# train_posneg[mask].shape[0]   # 893개 : 부정 : 별점 비율과 유사함  (부정 별점 1~4 : 895개, 1~3 : 335개)

## 데이터 증강 (부정 리뷰)

In [ ]:
train.head()

In [ ]:
# train= pd.read_csv(f"{DATA_PATH}train_posneg_done.csv")

In [ ]:
mask_neg = train['sentiment'] == '부정'
train[mask_neg].shape[0]

In [ ]:
mask_pos = train['sentiment'] == '긍정'
train[mask_pos].shape[0]

## 1) google 스토어리뷰 활용
- 구글 플레이스토어 배달의 민족 리뷰들 중에서 맛 관련, 배달,앱,결제 등 관련 단어 제외하고 10만개중 약 1500개 추출
- 부정비율이 16% 밖에 안됨. 부정 증강 용으로 사용은 부적절, 답변이 괜찮으므로 모범 답변으로 데이터에 더해서 활용

In [ ]:
google_food = pd.read_csv(f"{DATA_PATH}playstore_food_reply_0904.csv")

In [ ]:
google_train = google_food[['review','reply']]


- 긍부정비율 확인

In [ ]:
google_train_posneg = analyze_reviews(google_train['review'])

In [ ]:
tmp = google_train_posneg[['sentiment','score']]

In [ ]:
google_train = pd.concat([google_train,tmp],axis=1)
google_train

In [ ]:
mask_neg = google_train['sentiment'] == '부정'
google_train[mask_neg].shape[0]

In [ ]:
mask_pos = google_train['sentiment'] == '긍정'
google_train[mask_pos].shape[0]

In [ ]:
261/(1282+261)

In [ ]:
# 기존 train데이터와 구글 답변 데이터 concat

train = pd.concat([train,google_train],axis=0)
train.shape

## 2) resample 방법

In [ ]:
train

In [ ]:
mask_neg = train['sentiment'] == '부정'
train[mask_neg].shape[0]

In [ ]:
mask_pos = train['sentiment'] == '긍정'
train[mask_pos].shape[0]

In [ ]:
from sklearn.utils import resample

desired_samples = train[mask_pos].shape[0] - train[mask_neg].shape[0] #pos, neg개수 동일하게 하기 위해서
minority_class = train[train['sentiment'] == '부정']


In [ ]:
oversampled_data = resample(minority_class, replace=True, n_samples=desired_samples, random_state=42)
oversampled_data

In [ ]:
aug_train = pd.concat([train,oversampled_data],axis=0)
aug_train.shape

In [ ]:
aug_train.to_csv(f"{DATA_PATH}yogiyo_aug_train_token_masked.csv")



```
# 코드로 형식 지정됨
```



## 3) Paraphrase 사전학습모델 활용
- 약 700개의 데이터, 요기요 별점 1-4를 문장 재추출
- EDA대신 문장을 순서바꾸지 않으면서 증강 활용 가능. 그러나 어색한 문장 존재

In [ ]:
paraphrased[['review','reply']]


# 데이터 학습

## 사전학습모델

In [ ]:
# aug_train.to_csv(f"{DATA_PATH}yogiyo_aug_train_token_masked.csv")

In [ ]:
shuffle_aug_train = aug_train.sample(frac=1, random_state=42)
# train_ft = shuffle_aug_train[:3000].astype(str)
train_ft = shuffle_aug_train.astype(str)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "skt/kogpt2-base-v2"
# model_name = "EasthShin/Youth_Chatbot_Kogpt2-base"

model = AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          bos_token='</s>',
                                          eos_token='</s>',
                                          unk_token='<unk>',
                                          pad_token='<pad>',
                                          mask_token='<mask>',
                                          max_len=1024)

tokenizer.eos_token_id # eos 토큰의 idx 반환

In [ ]:
tokenizer.tokenize(train['review'][0])

## 데이터셋 만들기

In [ ]:
train_ft = train_ft[['review','reply']]

In [ ]:
# class ChatDataset(torch.utils.data.Dataset):
#     def __init__(self,df):
#         self.review = df["review"].tolist()
#         self.reply = df["reply"].tolist()
#         # self.store = df["store"].tolist()

#     def __len__(self):
#         return len(self.review)

#     def __getitem__(self,idx):
#         return "<w>" + self.store[idx] + "</s><q>" + self.review[idx] + "</s><a>" + self.reply[idx] + "</s>"
#                 # 임의의 sos 토큰 <q>와 eos 토큰 </s> 삽입

In [ ]:
class ChatDataset(torch.utils.data.Dataset):
    def __init__(self,df):
        self.review = df["review"].tolist()
        self.reply = df["reply"].tolist()

    def __len__(self):
        return len(self.review)

    def __getitem__(self,idx):
        return "<w>" + "</s><q>" + self.review[idx] + "</s><a>" + self.reply[idx] + "</s>"

In [ ]:
def collate_fn(batch):
    x = tokenizer(batch, return_tensors="pt", padding=True)
    return {"x":x}

In [ ]:
train_dt = ChatDataset(train_ft)

In [ ]:
train_dl = torch.utils.data.DataLoader(train_dt,batch_size=2,collate_fn=collate_fn)

In [ ]:
batch = next(iter(train_dl))
batch

In [ ]:
tokenizer.pad_token_id # 패딩토큰인 3번 빼고 계산해야겠군

## 학습 loop함수화

In [ ]:
def train_loop(dataloader,model,optimizer,loss_fn,device):
    epoch_loss = 0
    model.train()

    for batch in tqdm(dataloader):
        x = batch["x"].to(device)
        pred = model(**x).logits # batch, seq, 단어별 실수값
        n_class = pred.shape[-1] # 클래스 개수
        pred = pred[:,:-1] # eos 토큰 제외
        pred = pred.reshape(-1,n_class) # 2차원 형태로 변환

        tgt = x["input_ids"][:,1:]
        tgt = tgt.flatten() # 1차원 벡터 형태로 변환

        mask = tgt != 3 # 패딩토큰인 3번 빼고 계산해야겠군
        tgt = tgt[mask]
        pred = pred[mask]
        loss = loss_fn(pred,tgt)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    epoch_loss /= len(dataloader)
    return epoch_loss

## 학습하기

In [ ]:
batch_size = 2
epochs = 10
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
train_ft['review'] = train_ft['review'].astype(str)
train_ft['reply'] = train_ft['reply'].astype(str)
# train_ft['store'] = train_ft['store'].astype(str)

In [ ]:
is_holdout = False
reset_seeds(SEED)

model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=3e-5)

train_dt = ChatDataset(train_ft)
train_dl = torch.utils.data.DataLoader(train_dt,batch_size=batch_size,shuffle=True,collate_fn=collate_fn)

for i in range(epochs):
    train_loss = train_loop(train_dl, model, optimizer, loss_fn, device)
    print(train_loss)

    if is_holdout:
        break

## 모델 저장하기

In [ ]:
model.save_pretrained(f"{DATA_PATH}koGPT2_yogiyo+google_masktoken_full_0905") #0.3073174620668093

## 답변 생성

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# model_name = 'MLOpsEngineer/review-rate-prediction'

# model = AutoModelForCausalLM.from_pretrained(f"{DATA_PATH}ygy_cont3000_youth_ver1")
model = AutoModelForCausalLM.from_pretrained(f"{DATA_PATH}yogiyo_kogpt2_model0901")

In [ ]:
model_name = "skt/kogpt2-base-v2"

tokenizer = AutoTokenizer.from_pretrained(model_name, #해당 모델의 토크나이저
                                          bos_token='</s>',
                                          eos_token='</s>',
                                          unk_token='<unk>',
                                          pad_token='<pad>',
                                          mask_token='<mask>',
                                          max_len=1024,
                                          padding_side='left')

In [ ]:
def correct_spell(text):
    if isinstance(text, str):  # 입력 데이터가 문자열인 경우에만 맞춤법 검사 수행
        corrected_text = spell_checker.check(text).checked
        return corrected_text
    else:
        return text

In [ ]:
# @torch.no_grad()
# def chatbot(model,tokenizer,max_len,device):
#     model.eval()
#     while True:
#         text = input("소비자 > ").strip()
#         if text == "quit":
#             break
#         text = "<q>" + text + "</s><a>"
#         x = tokenizer.encode(text,return_tensors="pt").to(device) # batch, seq

#         result_ids = model.generate(x,
#                             max_length=max_len,
#                             repetition_penalty=2.0,
#                             use_cache=True,
#                             do_sample=True, # 확률적 샘플링 여부(False = greedy 방식)
#                             temperature=0.2, # 소프트맥스 온도(확률적 샘플링일 때만 작동)
#                             top_k=5) # 상위 확률 k개 기준으로 확률적 샘플링 하겠다(확률적 샘플링일 때만 작동)
#         q_len = len(text) + 1
#         text = tokenizer.decode(result_ids[0])

#         from hanspell import spell_checker

#         text = correct_spell(text)

#         # if len(text) > max_len:
#         #     text = text[:max_len]

#         print("사장님 > ",text[q_len:-4])

In [ ]:

# chatbot 함수 정의
def chatbot(model, tokenizer, max_len, review_list):
    responses = []  # 결과를 저장할 리스트 초기화

    for review in review_list:
        # 대화 시작 및 종료 토큰 추가
        text = "<q>" + review + "</s><a>"

        # 입력 텍스트를 인코딩
        x = tokenizer.encode(text, return_tensors="pt")
        # max_len = 10000, max_length=max_len

        # 모델로부터 응답 생성
        result_ids = model.generate(x,
                                    max_length=max_len,
                                    num_return_sequences=1,
                                    repetition_penalty=2.0,
                                    use_cache=True,
                                    do_sample=True, # 확률적 샘플링 여부(False = greedy 방식)
                                    temperature=0.2, # 소프트맥스 온도(확률적 샘플링일 때만 작동)
                                    top_k=5) # 상위 확률 k개 기준으로 확률적 샘플링 하겠다(확률적 샘플링일 때만 작동)




        # 응답을 텍스트로 디코딩하여 저장
        response = tokenizer.decode(result_ids[0], skip_special_tokens=True)

        response = response.split("<a>", 1)[1]
        response = correct_spell(response)
        responses.append(response)

    # 결과를 데이터프레임으로 변환
    result_df = pd.DataFrame({"Review": review_list, "Response": responses})
    return result_df


In [ ]:
review_list = [

    '맛이 있어요',
    '최고에요',
    '사장님이 친절하시네요',
    '맛은 있는데 배달이 식어서 왔어요ㅠㅠ',
    '맛이 없어요',
    '이 머리카락 들어있는거 뭔가요?',
    '배달이 너무 불친절해요',
    '최악이에요'

]

# chatbot 실행 및 결과 출력
result_df = chatbot(model, tokenizer, 1024, review_list)

pd.DataFrame(result_df)

In [ ]:
result_df.to_csv(f'{DATA_PATH}yogiyo_kogpt2_model0901_1.csv',index=False)

In [ ]:
review_list = [
    # 특정 메뉴명 포함 (맛)
    '이집 피자 최애 넘나 맛있어요',
    '치킨이 바삭바삭하니 맛있네요',
    '마라샹궈 넘넘 맛있고 양도 푸짐해용! 감사합니다!',
    '간만에 술한잔생각나서 연어 주문해서 먹는데 맛있네요^^ 양이 좀적어진거같은데 다음에는 추가로 시켜야 겠어요^^ 잘먹었습니당^^',
    '짜장면 배달이 너무 느리게 와서 음식이 다 불었네요..사장님 신경좀 써주세요',
    '쌀국수 먹는데 비닐 나왔네요 반품 이런건 안하지만 청결에 신경 써주세요',
    '요기 김치찜 맛있는데 가끔 좀 짜요ㅠㅠ다음엔 육수 추가로 주심 좋겠어요',

    # 메뉴의 특성 포함 (맛)
    '닭도 쫄깃하니 맛있고 떡볶이도 맛나요👍',
    '오늘은 비계가 많은거 같아요.. 양이 상대적으로 적어서 아쉬웠어요.',
    '크림이 너무 부드럽고 맛있어요, 또 시켜먹을게요!!',
    '맛있게 먹었어요 4단계로 해도 맵지는 않아요',


    # 배달 및 기타 코멘트
    '배달 빠르게 와서 맛있게 먹었습니다!',
    '완전 만족했어요 전문 블로거인데요 양쪼끔만 더 많았으면! 종종 시켜먹을게요',
    '맛은 정말 좋았지만, 편의점가려다 배달 도착한걸 발견 못해다면 다 상한 음식 먹을뻔했네요. 노크도 없고 문자도 안주시면 음식이 온지 어떻게 알으라는건지....',
    '.................',
    '벌레 나왔어요.. 다신 안시켜먹을 것 같아요',
]

result_df = chatbot(model, tokenizer, 128, review_list)

pd.DataFrame(result_df)

In [ ]:

chatbot(model,tokenizer,128,device) # quit

# 답변 평가

## PPL

In [ ]:
import torch
import torch.nn as nn

# # 가정: 언어 모델 클래스가 이미 정의되어 있다고 가정
# class LanguageModel(nn.Module):
#     # 모델 정의 ...

# 퍼플렉시티 계산 함수
def calculate_perplexity(model, dataloader, loss_function):
    total_loss = 0.0
    total_tokens = 0

    with torch.no_grad():
        for batch in dataloader:
            inputs, targets = batch
            outputs = model(inputs)
            loss = loss_function(outputs, targets)
            total_loss += loss.item()
            total_tokens += targets.numel()

    avg_loss = total_loss / total_tokens
    perplexity = 2 ** avg_loss  # 퍼플렉시티 계산

    return perplexity

# 사용 예시
# model = LanguageModel(...)  # 언어 모델 클래스 초기화
# dataloader = torch.utils.data.DataLoader(...)  # 데이터로더 초기화
# loss_function = nn.CrossEntropyLoss()  # 손실 함수 초기화
# perplexity = calculate_perplexity(model, dataloader, loss_function)
# print(f"Perplexity: {perplexity}")


## BLEU

## METEOR

## SSA